In [ ]:
!pip install -q numpy pandas scikit-learn tensorflow==2.14.0


In [2]:
import numpy as np
import pandas as pd
# Load existing "good" data
df = pd.read_csv("joint_angle_dataset.csv")

# Label original data as good form
df["label"] = 1

# Simulate bad form: random noise added to joint angles
noise = np.random.normal(loc=0.0, scale=10.0, size=df.shape)  # 10 degree std deviation
df_bad = df.copy()
df_bad.iloc[:, :4] += noise[:, :4]
df_bad["label"] = 0  # Label as bad form

# Combine good and bad data
df_full = pd.concat([df, df_bad], ignore_index=True)

# Shuffle
df_full = df_full.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Save combined dataset
df_full.to_csv("joint_angle_dataset_labeled.csv", index=False)

print(f"[✓] Good and fake-bad data saved to joint_angle_dataset_labeled.csv")



FileNotFoundError: [Errno 2] No such file or directory: 'joint_angle_dataset.csv'

In [ ]:
X = df[['left_shoulder_angle', 'left_elbow_angle', 'right_shoulder_angle', 'right_elbow_angle']]
y = df['label']  # 1 = good form, 0 = bad form (you may need to label this manually for now)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Input(shape=(4,)),
    layers.Dense(8, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=16
)


In [ ]:
# Save the model
model.save("form_classifier.keras")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("form_classifier.tflite", "wb") as f:
    f.write(tflite_model)
